## Throttle and Bandwith Demo

This notebook demonstrates the effect of throttling on effective bandwidth.
It uses [holoviews](http://holoviews.org/) to create an interactive histogram.

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from throttler.throttle import Throttle

The `make_histogram` method creates a histogram of bandwidth for each 100-microsecond window, based on
throttling a sample message stream with a parametrized `max_messages` and `time_window`.  The message stream
arrival times are modeled using a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution).

In [ ]:
def make_histogram(max_messages, time_window):
    throttle = Throttle(max_messages, time_window)
    times = np.cumsum(np.random.poisson(1, 100000))
    df = pd.DataFrame({'time': pd.to_datetime(times, unit='us'), 
                       'passed': [throttle.accept(t) for t in times]})
    bins=np.linspace(50,150,51)
    hist=np.histogram(df[df['passed']].groupby(pd.Grouper(key='time', freq='100us')).size(), bins=bins)
    return hv.Histogram(hist)

The next cell uses holoviews to create an interactive histogram based on the parameter `m`, which we use 
for both the `max_messages` and `time_window`.  Note that the special case `m==100` looks different because
there is a hard limit on bandwidth due to the throttle.

In [ ]:
%%opts Histogram [width=600 toolbar='above']
hv.HoloMap([(m, make_histogram(m,m)) for m in [2,3,5,10,20,50,100,200,500,1000]], kdims=['m'])